In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name

conf = (SparkConf().setMaster("k8s://https://192.168.219.100:6443") # Your master address name
        .set("spark.kubernetes.container.image", "joron1827/pyspark:v2") # Spark image name
        .set("spark.driver.port", "2222") # Needs to match svc
        .set("spark.driver.blockManager.port", "7777")
        .set("spark.driver.host", "driver-service.jupyterhub.svc.cluster.local") # Needs to match svc
        .set("spark.driver.bindAddress", "0.0.0.0")
        .set("spark.kubernetes.namespace", "spark")
        .set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
        .set("spark.kubernetes.authenticate.serviceAccountName", "spark")
        .set("spark.executor.instances", "4")
        .set("spark.kubernetes.container.image.pullPolicy", "IfNotPresent")
        .set("spark.app.name", "joronSpark")
        .set("spark.executor.cores", "4")
        .set("spark.executor.memory", "16g"))

In [2]:

spark = SparkSession.builder.config(conf=conf).getOrCreate()


/home/jovyan/.local/lib/python3.9/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


23/06/01 04:00:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_text = spark.read.parquet("hdfs://192.168.219.121:9000/crawling_stock_text_merge/text_merged.parquet")
df_text.show()

+------+----------------+------------------------------------+-----+---+---+-----------------+
|  code|            date|                               title|views|pos|neg|__index_level_0__|
+------+----------------+------------------------------------+-----+---+---+-----------------+
|005930|2023.04.06 14:35|                              여러분|  230|  0|  0|                0|
|005930|2023.04.06 14:34|     나라가  공매도  개망조 들었다. |  246|  1|  0|                0|
|005930|2023.04.06 14:34|     삼성전자 직원 5분의 1 감축 단행|  296|  5|  0|                0|
|005930|2023.04.06 14:33|                    분위기 좋고 좋고|  250|  1|  0|                0|
|005930|2023.04.06 14:33|        삼성전자는 허구한날 왜이래요|  268|  1|  0|                0|
|005930|2023.04.06 14:32|                         공매도 전자|  242|  1|  0|                0|
|005930|2023.04.06 14:32|     외국인 선물매도 2.1조 넘은거...|  292|  2|  0|                0|
|005930|2023.04.06 14:32|                    순종 못해 망했다|  233|  0|  0|                0|
|005930|2023.04.06 14:31|

In [4]:
df_stock=spark.read.parquet("hdfs://192.168.219.121:9000/stock_processed/stock_df_processed.parquet")
df_stock.show()

+--------+----+----+----+----+------+----------+------------------+------------+-----------------------+-----------------------+
|    날짜|시가|고가|저가|종가|거래량|stock_code|               RSI|이동평균_5일|Local_Maximum_5일_RSI70|Local_Minimum_5일_RSI30|
+--------+----+----+----+----+------+----------+------------------+------------+-----------------------+-----------------------+
|20170601|8230|8250|8080|8110|180501|    000390|              null|      8110.0|                   null|                   null|
|20170602|8180|8210|8110|8200| 44540|    000390|             100.0|      8155.0|                   null|                   null|
|20170605|8200|8240|8160|8170| 38293|    000390|             100.0|      8160.0|                 8160.0|                   null|
|20170607|8190|8190|8070|8070| 51072|    000390|              75.0|      8137.5|                   null|                   null|
|20170608|8180|8180|7750|7970|722148|    000390|40.909090909090914|      8104.0|                   null|             

In [5]:
from pyspark.sql.functions import col, to_date, regexp_replace, substring

# 텍스트 데이터의 "date" 컬럼에서 "-"를 제거하여 형식을 변환합니다.
temp_text = df_text.withColumn("date", substring(col("date"), 1, 10))
temp_text = temp_text.withColumn("date", to_date(regexp_replace(col("date"), "\\.", ""), "yyyyMMdd"))

temp_text.show()

+------+----------+------------------------------------+-----+---+---+-----------------+
|  code|      date|                               title|views|pos|neg|__index_level_0__|
+------+----------+------------------------------------+-----+---+---+-----------------+
|005930|2023-04-06|                              여러분|  230|  0|  0|                0|
|005930|2023-04-06|     나라가  공매도  개망조 들었다. |  246|  1|  0|                0|
|005930|2023-04-06|     삼성전자 직원 5분의 1 감축 단행|  296|  5|  0|                0|
|005930|2023-04-06|                    분위기 좋고 좋고|  250|  1|  0|                0|
|005930|2023-04-06|        삼성전자는 허구한날 왜이래요|  268|  1|  0|                0|
|005930|2023-04-06|                         공매도 전자|  242|  1|  0|                0|
|005930|2023-04-06|     외국인 선물매도 2.1조 넘은거...|  292|  2|  0|                0|
|005930|2023-04-06|                    순종 못해 망했다|  233|  0|  0|                0|
|005930|2023-04-06|                     삼성전자 1분기 |  305|  1|  1|                0|
|005930|

In [6]:
from pyspark.sql.functions import col, to_date, regexp_replace



# "date" 컬럼의 형식을 "yyyyMMdd"로 변환합니다.
temp_stock = df_stock.withColumn("날짜", to_date(regexp_replace(col("날짜"), "-", ""), "yyyyMMdd"))
temp_stock.show()

+----------+----+----+----+----+------+----------+------------------+------------+-----------------------+-----------------------+
|      날짜|시가|고가|저가|종가|거래량|stock_code|               RSI|이동평균_5일|Local_Maximum_5일_RSI70|Local_Minimum_5일_RSI30|
+----------+----+----+----+----+------+----------+------------------+------------+-----------------------+-----------------------+
|2017-06-01|8230|8250|8080|8110|180501|    000390|              null|      8110.0|                   null|                   null|
|2017-06-02|8180|8210|8110|8200| 44540|    000390|             100.0|      8155.0|                   null|                   null|
|2017-06-05|8200|8240|8160|8170| 38293|    000390|             100.0|      8160.0|                 8160.0|                   null|
|2017-06-07|8190|8190|8070|8070| 51072|    000390|              75.0|      8137.5|                   null|                   null|
|2017-06-08|8180|8180|7750|7970|722148|    000390|40.909090909090914|      8104.0|                   nu

In [7]:
from pyspark.sql.functions import col, when

# 텍스트 데이터와 주가 데이터를 조인하여 일치하는 행을 찾습니다.
joined_df = temp_text.join(temp_stock, (temp_text.date == temp_stock.날짜) & (temp_text.code == temp_stock.stock_code), "left_outer")

# 새로운 열을 추가합니다.
final_df = joined_df.withColumn("New_Column",
                                when(col("Local_Maximum_5일_RSI70").isNotNull(), 2)
                                .when(col("Local_Minimum_5일_RSI30").isNotNull(), 0)
                                .otherwise(1))

final_df.show()

+------+----------+-------------------------------------+-----+---+---+-----------------+----------+------+------+------+------+------+----------+-----------------+------------+-----------------------+-----------------------+----------+
|  code|      date|                                title|views|pos|neg|__index_level_0__|      날짜|  시가|  고가|  저가|  종가|거래량|stock_code|              RSI|이동평균_5일|Local_Maximum_5일_RSI70|Local_Minimum_5일_RSI30|New_Column|
+------+----------+-------------------------------------+-----+---+---+-----------------+----------+------+------+------+------+------+----------+-----------------+------------+-----------------------+-----------------------+----------+
|002600|2022-12-08|                          오늘 거래량|13347|  0|  0|                0|2022-12-08|201000|206000|198500|206000|  1620|    002600|71.92982456140349|    197400.0|                   null|                   null|         1|
|002600|2022-08-25|                                  1주|  431|  0|  0|       

In [53]:
from pyspark.sql import functions as F

# 데이터 프레임을 code 기준으로 그룹화하고 new_column에서 0, 1, 2의 개수를 추출합니다.
grouped_df = final_df.groupBy("code").agg(
    F.count(F.when(final_df.New_Column == 0, 1)).alias("count_0"),
    F.count(F.when(final_df.New_Column == 1, 1)).alias("count_1"),
    F.count(F.when(final_df.New_Column == 2, 1)).alias("count_2")
)

grouped_df.show()

+------+-------+-------+-------+
|  code|count_0|count_1|count_2|
+------+-------+-------+-------+
|074610|   2360| 171747|   2959|
|008930|    384|  33039|    655|
|003850|    358|  33628|    579|
|000910|    148|  45370|    741|
|003060|    762|  73617|    541|
|009460|    756|  90277|   2225|
|010060|   1250|  73999|    558|
|011930|    925|  96516|   1679|
|009830|   1754| 160440|   3512|
|010820|    228|  52492|   1669|
|016360|    347|  37830|    344|
|002630|    408|  41407|    773|
|011390|    874|  41836|   1487|
|002900|    272|  42169|    772|
|013000|    130|  40818|    911|
|013700|    934|  44711|    911|
|012690|    232|  49230|   2279|
|010660|    130|  61921|   2926|
|019170|  11827| 881926|  37456|
|001440|   1030| 148740|   4079|
+------+-------+-------+-------+
only showing top 20 rows



In [8]:

simple_text_df = final_df.select("code", "date", "title", "New_Column")
simple_text_df.count()
                                         
## 30048375 결과

30048375

In [9]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# 정규식 패턴을 이용한 데이터 전처리 함수 정의
def preprocess_text(text):
    cleaned_text = re.sub(r'[^ ㄱ-ㅎ가-힣 ]+', '', text)
    return cleaned_text

# UDF(사용자 정의 함수) 등록
preprocess_udf = udf(preprocess_text, StringType())

# 전처리 함수를 적용하여 새로운 컬럼 생성
preprocessed_df = simple_text_df.withColumn("title", preprocess_udf("title"))

# 결과 확인
preprocessed_df.show()

+------+----------+------------------------------------+----------+
|  code|      date|                               title|New_Column|
+------+----------+------------------------------------+----------+
|005930|2023-04-06|                              여러분|         1|
|005930|2023-04-06|      나라가  공매도  개망조 들었다 |         1|
|005930|2023-04-06|       삼성전자 직원 분의  감축 단행|         1|
|005930|2023-04-06|                    분위기 좋고 좋고|         1|
|005930|2023-04-06|        삼성전자는 허구한날 왜이래요|         1|
|005930|2023-04-06|                         공매도 전자|         1|
|005930|2023-04-06|외국인 선물매도 조 넘은거 처음보는데|         1|
|005930|2023-04-06|                    순종 못해 망했다|         1|
|005930|2023-04-06|                      삼성전자 분기 |         1|
|005930|2023-04-06|                  공매도 잘 생각해라|         1|
|005930|2023-04-06|                     내일 천원 이상 |         1|
|005930|2023-04-06|            감산 안하다고 뻘짓하더니|         1|
|005930|2023-04-06|                              성렬아|         1|
|005930|2023-04-06|  

In [10]:
file_path = 'stopwords.txt'  # 파일 경로 설정

with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    stopword_list = [line.strip() for line in lines]
    
print(stopword_list)

['가', '가까스로', '가령', '각', '각각', '각자', '각종', '갖고말하자면', '같다', '같이', '개의치않고', '거니와', '거바', '거의', '것', '것과 같이', '것들', '게다가', '게우다', '겨우', '견지에서', '결과에 이르다', '결국', '결론을 낼 수 있다', '겸사겸사', '고려하면', '고로', '곧', '공동으로', '과', '과연', '관계가 있다', '관계없이', '관련이 있다', '관하여', '관한', '관해서는', '구', '구체적으로', '구토하다', '그', '그들', '그때', '그래', '그래도', '그래서', '그러나', '그러니', '그러니까', '그러면', '그러므로', '그러한즉', '그런 까닭에', '그런데', '그런즉', '그럼', '그럼에도 불구하고', '그렇게 함으로써', '그렇지', '그렇지 않다면', '그렇지 않으면', '그렇지만', '그렇지않으면', '그리고', '그리하여', '그만이다', '그에 따르는', '그위에', '그저', '그중에서', '그치지 않다', '근거로', '근거하여', '기대여', '기점으로', '기준으로', '기타', '까닭으로', '까악', '까지', '까지 미치다', '까지도', '꽈당', '끙끙', '끼익', '나', '나머지는', '남들', '남짓', '너', '너희', '너희들', '네', '넷', '년', '논하지 않다', '놀라다', '누가 알겠는가', '누구', '다른', '다른 방면으로', '다만', '다섯', '다소', '다수', '다시 말하자면', '다시말하면', '다음', '다음에', '다음으로', '단지', '답다', '당신', '당장', '대로 하다', '대하면', '대하여', '대해 말하자면', '대해서', '댕그', '더구나', '더군다나', '더라도', '더불어', '더욱더', '더욱이는', '도달하다', '도착하다', '동시에', '동안', '된바에야', '된이상', '두번째로', '둘', '둥둥', '뒤따라', '뒤이어'

In [12]:
from pyspark.sql.functions import *
from pyspark.sql.types import Row
from konlpy.tag import Okt




def extract_nouns(content):
    okt = Okt()
    tokens = okt.morphs(content)
    tokens = [token for token in tokens if token not in stopword_list]
    
    return tokens

extract_nouns_udf = udf(extract_nouns, ArrayType(StringType()))

tokenized_df = preprocessed_df.withColumn("tokens", extract_nouns_udf("title"))

tokenized_df.show()


+------+----------+------------------------------------+----------+----------------------------+
|  code|      date|                               title|New_Column|                      tokens|
+------+----------+------------------------------------+----------+----------------------------+
|005930|2023-04-06|                              여러분|         1|                          []|
|005930|2023-04-06|      나라가  공매도  개망조 들었다 |         1|[나라, 공매도, 개, 망조, ...|
|005930|2023-04-06|       삼성전자 직원 분의  감축 단행|         1|[삼성, 직원, 분, 감축, 단행]|
|005930|2023-04-06|                    분위기 좋고 좋고|         1|        [분위기, 좋고, 좋고]|
|005930|2023-04-06|        삼성전자는 허구한날 왜이래요|         1|  [성, 는, 허구, 한, 날, 요]|
|005930|2023-04-06|                         공매도 전자|         1|                    [공매도]|
|005930|2023-04-06|외국인 선물매도 조 넘은거 처음보는데|         1|[외국인, 선물, 매도, 조, ...|
|005930|2023-04-06|                    순종 못해 망했다|         1|      [순종, 못, 해, 망했다]|
|005930|2023-04-06|                      삼성전자 분기 |       

In [13]:

# 선택할 종목 코드
selected_stock_code = "005930"

# 특정 종목 코드에 대한 RSI 값 추출
selected_df = tokenized_df.filter(tokenized_df.code == selected_stock_code).select('tokens')


pandas_df = selected_df.toPandas()

from collections import Counter
count = Counter(pandas_df['nouns'][:10000])

count 




----------------------------------------                                        
Exception occurred during processing of request from ('127.0.0.1', 41166)
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/jovyan/.local/lib/python3.9/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/home/jovyan/.local/lib/python3.9/site-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/home/jovyan/.local/lib/python3.9/site-packages/pyspark/accumula

ValueError: Unable to parse datatype from schema. An error occurred while calling o138.schema

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
# from matplotlib import font_manager, rc
# font_path = "c:/Windows/fonts/malgun.ttf"
# font_name = font_manager.FontProperties(fname = font_path).get_name()
# matplotlib.rc('font', family=font_name)

plt.figure(figsize=(12,5))
plt.xlabel('키워드')
plt.ylabel('빈도수')
plt.grid(True)

sorted_Keys = sorted(word_count, key=word_count.get, reverse=True)
sorted_Values = sorted(word_count.values(), reverse=True)

plt.bar(range(len(word_count)), sorted_Values, align='center')
plt.xticks(range(len(word_count)), list(sorted_Keys), rotation='vertical')

plt.show()

In [14]:
spark.stop()

ConnectionRefusedError: [Errno 111] Connection refused